In [ ]:
# -*- coding: utf-8 -*-
"""
一个基于大型语言模型 (glm-4-flash-250414) 和 Function Calling 的电路设计 Agent (V4.3 - 修正版 Hyper Verbose Demo)。
这个 Agent 能够理解用户的自然语言指令，并通过调用预定义的工具函数来模拟电路的添加、连接、描述和清空等操作。
本版本修复了 Agent 内部方法名称与工具定义名称不匹配的问题，确保工具调用能被正确路由和执行。
更重要的是，根据用户需求，调整了 Agent 的行为模式：
1. 融入了“顶尖程序员、家庭经济支柱、必须保住工作”的角色设定。
2. 强调了对用户（“老板”）需求的认真确认、积极主动和提供完美方案的态度。
3. **核心功能：在生成正式回复前，强制模型模拟一段详细的内部思考过程，并用 <think> 和 </think> 标签包裹。**

代码保留了极其详细的日志输出，展示 Agent 内部处理、状态变化和 LLM 交互的每一个细节，
包括 API 调用计时、Token 使用统计等，非常适合学习和调试 Function Calling Agent 的工作原理。
V4.1 修正了处理 Function Calling 时消息历史传递的问题。
V4.2 修正了调用 model_dump_json 时因库版本更新导致 ensure_ascii 参数不再支持的 TypeError。
V4.3 修正了 Agent 内部工具方法名称与工具定义名称的匹配问题，并实现了新的角色和思考模式。
"""

import re               # 正则表达式库，用于验证元件 ID 格式（确保以字母开头，后跟字母数字）
import os               # 操作系统库，主要用于访问环境变量 (API Key)
import json             # JSON 库，用于解析 LLM 返回的参数和打印复杂的字典/对象
import time             # 时间库，用于测量 API 调用的耗时
from zhipuai import ZhipuAI # 导入智谱 AI 的官方 Python SDK
import traceback        # 用于在捕获异常时打印详细的错误堆栈信息（帮助定位问题）
# 如果使用了 Pydantic V2+，model_dump 方法不再支持 ensure_ascii 参数，需要捕获 TypeError
# from pydantic import VERSION as PYDANTIC_VERSION # 可以用来检查 Pydantic 版本

# --- 电路元件类 (CircuitComponent Class) ---
# 定义电路的基本构建单元：元件。
class CircuitComponent:
    """
    代表电路中的一个基本元件。
    负责存储元件的 ID、类型和可选的值。
    """
    def __init__(self, component_id, component_type, value=None):
        """
        初始化一个电路元件实例。

        Args:
            component_id (str): 元件的唯一标识符 (例如 "R1", "C2", "U1")。必须是有效的非空字符串。
                                  会在内部转换为大写存储。
            component_type (str): 元件的类型 (例如 "电阻", "电容", "LED")。必须是有效的非空字符串。
            value (str, optional): 元件的可选值 (例如 "1kOhm", "10uF", "9V")。默认为 None。
                                   如果提供，会被清理（去除首尾空格），空字符串也会被视为 None。

        Raises:
            ValueError: 如果 component_id 或 component_type 不是有效的非空字符串。
        """
        # 使用更深的缩进打印内部类方法的日志，以区分其调用层级
        print(f"        [Component Init] 开始初始化 CircuitComponent 实例...")
        print(f"            [Component Init]   - 接收ID: {component_id}, 类型: {component_type}, 值: {value}")

        # 输入验证：确保 ID 和类型是有效的非空字符串
        if not isinstance(component_id, str) or not component_id:
            print(f"            [Component Init] 错误: 元件ID无效。")
            raise ValueError("元件ID必须是有效的非空字符串")
        if not isinstance(component_type, str) or not isinstance(component_type, str): # 修正了类型判断，确保是字符串
            print(f"            [Component Init] 错误: 元件类型无效。")
            raise ValueError("元件类型必须是有效的非空字符串")

        # 将 ID 统一转换为大写，便于后续不区分大小写的查找和比较
        self.id = component_id.upper()
        print(f"            [Component Init]   - 内部存储ID (大写): '{self.id}'")
        # 存储元件类型
        self.type = component_type
        print(f"            [Component Init]   - 内部存储类型: '{self.type}'")

        # 清理和标准化元件值
        original_value = value # 记录原始值用于日志对比
        if value is not None and not isinstance(value, str):
           # 如果值不是字符串，尝试转换为字符串。例如 LLM 返回数字 1000，我们希望存储为字符串 "1000"
           try:
                value = str(value)
                print(f"    [Agent Internals:CircuitComponent init]   - 接收值非字符串，尝试转为字符串: '{value}' (原始值: {original_value})")
           except Exception as e:
               # 转换失败则打印警告，并继续（值可能为 None 或保持原样）
               print(f"    [Agent Internals:CircuitComponent init] 警告: 无法将接收值 {repr(original_value)} 转换为字符串，将保持原样或 None。")
               value = original_value # 转换失败则恢复原始值

        if isinstance(value, str):
            value = value.strip() # 去除首尾可能存在的空白符
            value = value if value else None # 如果清理后为空字符串，则视作 None (无值)
        # 如果值经过清理发生了变化，打印出来便于调试
        # 仅当原始值非 None 且清理后值与原始值不同时打印变化
        if original_value is not None and value != original_value:
             print(f"    [Agent Internals:CircuitComponent init]   - 清理后的值为: {repr(value)} (原始值: {repr(original_value)})")


        print(f"        [Component Init] CircuitComponent 实例 '{self.id}' 初始化完成。")

    def __str__(self):
        """
        返回元件的易于阅读的字符串表示形式，主要用于向用户展示。
        例如: "元件: 电阻 (ID: R1) (值: 1kOhm)" 或 "元件: LED (ID: L1)"
        """
        # 如果元件有值，则在字符串中包含值的表示
        value_str = f" (值: {self.value})" if self.value is not None else ""
        # 格式化输出字符串
        return f"元件: {self.type} (ID: {self.id}){value_str}"

    def __repr__(self):
        """
        返回元件的开发者友好的、明确的字符串表示形式，主要用于调试和日志记录。
        使用 repr(self.value) 来确保值的类型也被清晰表示 (例如 '1kOhm' vs 1000)。
        例如: "CircuitComponent(id='R1', type='电阻', value='1kOhm')"
        """
        # 返回一个能明确表示对象状态的字符串，适合开发者查看
        return f"CircuitComponent(id='{self.id}', type='{self.type}', value={repr(self.value)})"

# --- 电路设计 Agent 类 (CircuitDesignAgent Class) ---
# 核心类，封装了 Agent 的状态、工具、与 LLM 的交互逻辑以及执行操作的方法。
class CircuitDesignAgent:
    """
    基于 LLM (glm-4-flash-250414) 和 Function Calling 的电路设计 Agent (修正版超详细演示 V4.3)。
    能够理解自然语言指令，通过调用内部工具函数来管理电路设计，并提供极其详细的运行日志。
    此版本加入了程序员角色和强制思考模式。
    """
    def __init__(self, api_key, model_name="glm-4-flash-250414"):
        """
        初始化 Agent 实例。

        Args:
            api_key (str): 用于访问 Zhipu AI API 的密钥。
            model_name (str): 指定要使用的 LLM 模型名称。默认为 glm-4-flash-250414。

        Raises:
            ValueError: 如果未提供 API Key。
            ConnectionError: 如果初始化 Zhipu AI 客户端失败（例如 API Key 无效或网络问题）。
        """
        # 使用 '=' 分隔符和标题，清晰标记 Agent 初始化过程的开始
        print(f"\n{'='*30} 开始初始化 CircuitDesignAgent {'='*30}")
        print(f"[Agent Init] 版本: V4.3 Hyper Verbose Demo")
        print(f"[Agent Init] 目标模型: {model_name}")

        # 检查 API Key 是否提供，未提供则报错并终止初始化
        if not api_key:
            print("[Agent Init] 错误：未提供 API Key。初始化失败。")
            raise ValueError("必须提供 Zhipu AI API Key")
        self.api_key = api_key # 存储 API Key

        # --- Agent 内部状态 ---
        # 使用字典存储元件，键是元件ID (大写)，值是 CircuitComponent 对象
        # 方便通过 ID 快速查找元件
        self.components = {}
        # 使用集合存储连接关系，每个元素是一个按字母顺序排序后的元件ID元组 (例如 ('B1', 'R1'))
        # 集合能自动处理重复添加，排序后的元组确保 ('R1', 'B1') 和 ('B1', 'R1') 被视为相同连接
        self.connections = set()
        # 字典，用于为各种元件类型维护一个计数器，以便自动生成唯一的 ID (例如 R1, R2, C1...)
        # 键是类型代码 (如 'R'), 值是当前该类型的最高编号 (例如，如果已经有 R1, R2, 计数器应为 2)
        self._component_counters = {
            'R': 0, 'L': 0, 'B': 0, 'S': 0, 'C': 0, 'V': 0, 'G': 0, 'U': 0, 'Other': 0,
            # 可以在这里添加更多预定义的类型代码及其初始计数器
        }
        # 消息历史列表，用于存储用户和 Agent (包括工具调用和工具结果) 之间的对话，
        # 在多轮交互中作为上下文提供给 LLM。
        self.messages = []
        # 存储使用的模型名称
        self.model_name = model_name

        print("[Agent Init] 内部状态 (components, connections, counters, messages) 已初始化。")

        # --- 初始化 Zhipu AI 客户端 ---
        try:
            print("[Agent Init] 尝试使用提供的 API Key 连接 Zhipu AI 服务...")
            # 创建 ZhipuAI 客户端实例，用于后续与 LLM API 的交互
            # API Key 会在实例化时由 SDK 自动处理认证
            self.client = ZhipuAI(api_key=self.api_key)
            # (可选测试) 可以取消下面这行的注释来验证 API Key 和网络连接
            # print("[Agent Init] 测试 API 连接 (调用 models.list())...")
            # self.client.models.list() # 如果 Key 无效或网络不通，这里通常会抛出异常，被 except 捕获
            print("[Agent Init] Zhipu AI 客户端初始化成功。")
        except Exception as e:
            # 捕获初始化过程中可能发生的任何异常 (如认证失败、网络错误、API Key 格式问题)
            print(f"[Agent Init] 错误：初始化 Zhipu AI 客户端失败: {e}")
            # 打印更详细的错误堆栈，便于诊断
            traceback.print_exc()
            print("[Agent Init] 请检查 API Key 是否有效以及网络连接。")
            # 抛出 ConnectionError，通知调用者初始化失败
            raise ConnectionError(f"无法初始化 Zhipu AI 客户端: {e}") from e

        # --- 定义 Agent 可用的工具 (Functions/Tools) ---
        # 这是 Function Calling 的核心。我们在这里定义 Agent 能“告诉” LLM 它会做的具体事情。
        # LLM 会根据用户请求和这里的描述，决定调用哪个工具以及提取哪些参数。
        # 这些工具的 'name' 字段必须与 Agent 类中实际执行功能的 **方法名称** 完全匹配。
        # 我们在 V4.3 版本中修正了这里的方法名称匹配问题。
        self.tools = [
            {
                "type": "function", # 指明这是一个函数/工具类型的定义
                "function": {
                    "name": "add_component_tool", # 工具的唯一名称，与下方方法名一致
                    "description": "向电路中添加一个新的元件，如电阻、电容、电池、LED、开关、芯片等。如果用户没有指定ID，系统会自动生成一个唯一的ID。如果用户没有指定值，则值为空。", # 工具功能的清晰描述，LLM 会根据这个描述来理解工具的作用
                    "parameters": { # 定义这个工具需要哪些参数
                        "type": "object", # 参数是一个 JSON 对象
                        "properties": { # 对象包含的属性 (即参数)
                            "component_type": { # 参数1：元件类型
                                "type": "string", # 参数类型：字符串
                                "description": "要添加的元件类型，例如 '电阻', 'LED', '9V电池', '电容', '开关', '芯片', '接地' 等。" # 参数描述，告诉 LLM 如何从用户输入中提取这个参数
                            },
                            "component_id": { # 参数2：元件 ID (可选)
                                "type": "string", # 参数类型：字符串
                                "description": "用户为元件指定的可选ID (例如 R1, C_Input)。如果用户未指定，请不要提供此参数（或设为 null/空字符串），系统会为您自动生成。" # 描述中强调了可选性，并给出例子，并说明系统会自动生成ID
                            },
                            "value": { # 参数3：元件值 (可选)
                                "type": "string", # 参数类型：字符串
                                "description": "元件的可选值，例如 '1k' (电阻), '10uF' (电容), '9V' (电池)。请尽量提取用户提到的具体值。如果用户未指定，请不要提供此参数或设为 null。" # 描述中强调了可选性，并给出例子
                            }
                        },
                        "required": ["component_type"] # 明确指出哪些参数是必须的。这里只有元件类型是必须的。
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "connect_components_tool", # 工具名称，与下方方法名一致
                    "description": "连接电路中两个已存在的元件。需要提供两个元件的ID。", # 描述中强调了元件必须已存在
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "component_id_1": {
                                "type": "string",
                                "description": "第一个要连接的元件的ID (例如 'R1')。请确保提取正确的ID。" # 给出ID例子，并强调提取正确性
                            },
                            "component_id_2": {
                                "type": "string",
                                "description": "第二个要连接的元件的ID (例如 'B1')。请确保提取正确的ID。" # 给出ID例子，并强调提取正确性
                            }
                        },
                        "required": ["component_id_1", "component_id_2"] # 连接操作必须提供两个元件的ID
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "describe_circuit_tool", # 工具名称，与下方方法名一致
                    "description": "获取当前电路的详细描述，包括所有元件及其值，以及它们之间的连接关系。当用户询问当前电路状态时调用此工具。", # 描述何时调用
                    "parameters": { # 这个工具不需要参数
                        "type": "object", # 参数类型仍然是 object
                        "properties": {} # 但 properties 为空对象
                    }
                    # "required" 字段可以省略，因为没有 properties
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "clear_circuit_tool", # 工具名称，与下方方法名一致
                    "description": "清空当前的电路设计，删除所有元件和连接，重置状态。用户明确要求重新开始或清空时调用此工具。", # 描述何时调用
                    "parameters": { # 这个工具也不需要参数
                        "type": "object",
                        "properties": {}
                    }
                }
            }
            # 未来可以添加更多工具，例如：
            # - "delete_component_tool": 删除指定 ID 的元件 (需要考虑处理相关连接)
            # - "update_component_value_tool": 修改指定 ID 元件的值
            # - "find_component_tool": 查询具有特定类型或值的元件
            # - "simulate_circuit_tool": 模拟电路行为（复杂，需要专门库）
            # - "save_circuit_tool": 保存电路设计到文件
        ]
        # 初始化完成前，打印定义的工具数量和结构，便于调试确认
        print(f"[Agent Init] 定义了 {len(self.tools)} 个可用工具 (Functions):")
        # 使用 json.dumps 美化打印工具定义，ensure_ascii=False 保证中文正常显示，indent=2 增加可读性
        try:
            print(json.dumps(self.tools, indent=2, ensure_ascii=False))
        except TypeError: # 捕获 Pydantic V2+ 中 ensure_ascii 参数不再支持的 TypeError
            print("[Agent Init] Warning: Pydantic version does not support ensure_ascii=False in json.dumps. Printing without it.")
            print(json.dumps(self.tools, indent=2))
        except Exception as e:
            print(f"[Agent Init] Warning: Failed to dump tools list to JSON: {e}. Printing raw list.")
            print(self.tools)

        print(f"\n{'='*30} CircuitDesignAgent 初始化完成 {'='*30}\n")


    # --- 内部核心操作方法 (名称与工具定义名称一致) ---
    # 这些方法是 Agent 实际执行电路操作的函数。
    # 它们会被 LLM 通过 Function Calling 间接调用（即 LLM 请求调用，Agent 内部路由到这些方法）。
    # 每个方法都包含详细的日志输出。这些方法的名称现在与 self.tools 列表中的 function.name 字段完全一致。

    def _generate_id(self, component_type):
        """
        (内部辅助方法) 根据元件类型生成一个唯一的、递增的元件ID。
        例如，对于 "电阻"，会依次生成 "R1", "R2", ...
        旨在提供一种在用户未指定 ID 时自动命名元件的方式。

        Args:
            component_type (str): 请求生成 ID 的元件类型 (例如 "电阻", "LED")。

        Returns:
            str: 生成的唯一元件 ID (例如 "R1")。
        """
        # 使用更深的缩进打印内部方法的日志，以区分其调用层级
        print(f"    [Agent Internals:_generate_id] 尝试为类型 '{component_type}' 生成 ID...")
        # 定义一个映射，将常见的元件类型关键字（中文、英文、单位等）映射到一个简短的类型代码
        # 这样可以使得相似类型的元件（如“电池”和“电源”）使用相同的代码前缀（如 'B'）
        # 这个映射可以根据实际需求扩展和优化，使其更全面和准确。
        type_map = {
            "电阻": "R", "resistor": "R", "ohm": "R", "欧姆": "R", # 电阻相关
            "led": "L", "发光二极管": "L",                          # LED 相关
            "电池": "B", "电源": "B", "battery": "B", "伏": "B", "v": "B", "电压": "B", # 电池/电源相关 (注意 'V' 的潜在冲突)
            "开关": "S", "switch": "S",                            # 开关相关
            "电容": "C", "capacitor": "C", "farad": "C", "法拉": "C", # 电容相关
            "电压源": "V", "voltage source": "V",                  # 电压源 (独立于电池)
            "地": "G", "接地": "G", "ground": "G",                  # 接地相关
            "芯片": "U", "集成电路": "U", "ic": "U"                   # 芯片/IC 相关
            # 可以根据需要扩展更多类型
        }
        # 默认的代码，用于在 type_map 中找不到匹配关键字的类型
        type_code = "Other"
        # 将输入类型转为小写，进行不区分大小写的匹配
        lower_type = component_type.lower()
        print(f"        [Agent Internals:_generate_id]   - 输入类型转换为小写: '{lower_type}'")
        # 遍历映射表，查找第一个匹配的关键字
        for keyword, code in type_map.items():
            # 如果关键字出现在小写的类型字符串中
            if keyword in lower_type:
                type_code = code # 找到匹配，使用对应的代码
                print(f"        [Agent Internals:_generate_id]   - 匹配到关键字 '{keyword}', 使用代码 '{type_code}'。")
                break # 找到第一个匹配就停止，避免将一个类型（如“电压源”）匹配到多个代码
        # 如果循环结束仍未找到匹配
        if type_code == "Other":
             print(f"        [Agent Internals:_generate_id]   - 未匹配到特定关键字，使用默认代码 'Other'。")

        # 获取当前类型代码的计数器值 (即上一个生成的编号)
        # 如果该 type_code 第一次出现 (例如 type_map 中没有，但用户指定了新类型)，需要在计数器字典中初始化它
        if type_code not in self._component_counters:
             self._component_counters[type_code] = 0
             print(f"        [Agent Internals:_generate_id]   - 新的类型代码 '{type_code}'，计数器初始化为 0。")

        initial_counter = self._component_counters[type_code]
        # 计数器加 1，得到新的编号
        self._component_counters[type_code] += 1
        # 生成新的 ID (例如 R1, R2, ...)
        new_id = f"{type_code}{self._component_counters[type_code]}"
        print(f"        [Agent Internals:_generate_id]   - 基于计数器 ({initial_counter} -> {self._component_counters[type_code]}), 初始生成 ID: '{new_id}'。")

        # 检查生成的 ID 是否已经存在于 self.components 字典中
        # 虽然理论上递增不应重复，但以防万一 (例如手动添加了 R5 后自动生成也到了 R5，或重置计数器后)
        while new_id in self.components:
            print(f"        [Agent Internals:_generate_id]   - ID '{new_id}' 已存在 (可能由手动指定或其他原因导致)，递增计数器并重新生成...")
            initial_counter = self._component_counters[type_code] # 记录当前计数器
            self._component_counters[type_code] += 1 # 再次增加计数器
            new_id = f"{type_code}{self._component_counters[type_code]}" # 重新生成 ID
            print(f"        [Agent Internals:_generate_id]   - 基于计数器 ({initial_counter} -> {self._component_counters[type_code]}), 重新生成 ID: '{new_id}'。")

        # 确认 ID 唯一后返回
        print(f"    [Agent Internals:_generate_id] 最终确定唯一 ID: '{new_id}'。")
        return new_id

    # 修改方法名称，与工具定义中的 name 一致
    def add_component_tool(self, component_type, component_id=None, value=None):
        """
        (内部执行方法) 向电路中添加一个新元件。
        这个方法由 LLM 请求调用 "add_component_tool" 时触发，并接收 LLM 提取的参数。

        Args:
            component_type (str): 元件类型 (来自 LLM 提取的参数)。
            component_id (str, optional): 用户指定的 ID (来自 LLM 提取的参数)。如果为 None 或无效，将自动生成。
            value (str, optional): 元件的值 (来自 LLM 提取的参数)。

        Returns:
            str: 操作结果的描述信息 (例如 "成功添加元件..." 或 "错误：...")，这个信息会返回给 LLM。
        """
        print(f"    [Agent Internals:add_component_tool] 开始处理添加元件请求 (由 'add_component_tool' 触发)...")
        print(f"        [Agent Internals:add_component_tool]   - 接收类型: {component_type}")
        print(f"        [Agent Internals:add_component_tool]   - 接收ID: {component_id}")
        print(f"        [Agent Internals:add_component_tool]   - 接收值: {value}")

        # 基本验证：元件类型必须提供且为字符串
        if not component_type or not isinstance(component_type, str):
            msg = "错误：添加元件失败，必须提供有效的元件类型（字符串）。这是非常严重的错误，请检查LLM提取的参数是否有问题！" # 加入程序员角色语气
            print(f"    [Agent Internals:add_component_tool] {msg}")
            return msg # 返回错误信息给 LLM

        target_id = None # 用于存储最终确定使用的元件 ID
        id_was_generated = False # 标记 ID 是否是自动生成的，用于后续返回信息

        # 处理用户指定的 ID (如果 LLM 提供了 component_id 且是有效的字符串)
        if component_id and isinstance(component_id, str):
            print(f"    [Agent Internals:add_component_tool] 检查用户提供的 ID '{component_id}'...")
            # 验证 ID 格式：必须以字母开头，后面可以跟字母或数字
            # 这是常见的元件 ID 命名约定 (如 R1, C10, U2A)
            if re.match(r'^[a-zA-Z][a-zA-Z0-9]*$', component_id):
                target_id = component_id.upper() # 格式有效，转换为大写进行内部存储和比较
                print(f"        [Agent Internals:add_component_tool]   - 用户提供ID格式有效，转换为大写: '{target_id}'。")
                # 检查 ID 是否已被占用
                if target_id in self.components:
                    msg = f"错误：添加元件失败，元件ID '{target_id}' 已被占用。老板，我们不能使用重复的ID，这会导致电路混乱！请尝试使用其他ID或不指定ID让我自动生成一个。" # 加入程序员角色语气
                    print(f"    [Agent Internals:add_component_tool] {msg}")
                    return msg # 如果 ID 已存在，返回错误信息
                else:
                    print(f"        [Agent Internals:add_component_tool]   - 用户提供ID '{target_id}' 可用。太好了！") # 加入程序员角色语气
            else:
                # 如果用户提供的 ID 格式无效 (例如 "1R", "R-1")
                msg_warning = f"用户提供的ID '{component_id}' 格式无效 (必须字母开头，后跟字母数字)。老板，这个ID格式不符合规范，我将忽略并为您自动生成一个新ID。" # 加入程序员角色语气
                print(f"        [Agent Internals:add_component_tool]   - {msg_warning}")
                target_id = None # 将 target_id 设回 None，以触发下面的自动生成逻辑
                # 注意：这里我们不立即返回错误，而是尝试自动生成 ID，提供更强的容错能力。

        # 如果经过上述处理后 target_id 仍然是 None (即用户未提供有效 ID 或格式错误被忽略)
        if target_id is None:
            print(f"    [Agent Internals:add_component_tool] 调用 _generate_id 为类型 '{component_type}' 生成ID...")
            target_id = self._generate_id(component_type) # 调用内部方法生成 ID
            id_was_generated = True # 标记 ID 是自动生成的
            print(f"        [Agent Internals:add_component_tool]   - 自动生成ID为: '{target_id}'。")

        # 清理和验证元件值 (与 CircuitComponent 的 __init__ 逻辑类似，确保一致性)
        original_value = value # 记录原始值用于日志对比
        if value is not None and not isinstance(value, str):
           # 如果值不是字符串，尝试转换为字符串。例如 LLM 返回数字 1000，我们希望存储为字符串 "1000"
           try:
                value = str(value)
                print(f"    [Agent Internals:add_component_tool]   - 接收值非字符串，尝试转为字符串: '{value}' (原始值: {original_value})")
           except Exception as e:
               # 转换失败则打印警告，并继续（值可能为 None 或保持原样）
               print(f"    [Agent Internals:add_component_tool] 警告: 无法将接收值 {repr(original_value)} 转换为字符串，将保持原样或 None。")
               value = original_value # 转换失败则恢复原始值

        if isinstance(value, str):
            value = value.strip() # 去除首尾可能存在的空白符
            value = value if value else None # 如果清理后为空字符串，则视作 None (无值)
        # 如果值经过清理发生了变化，打印出来便于调试
        # 仅当原始值非 None 且清理后值与原始值不同时打印变化
        if original_value is not None and value != original_value:
             print(f"    [Agent Internals:add_component_tool]   - 清理后的值为: {repr(value)} (原始值: {repr(original_value)})")


        # 尝试创建 CircuitComponent 实例并添加到电路状态中
        try:
            print(f"    [Agent Internals:add_component_tool] 尝试创建 CircuitComponent 对象 (ID='{target_id}', Type='{component_type}', Value={repr(value)})...")
            # 使用最终确定的 ID、类型和处理后的值创建元件对象
            new_component = CircuitComponent(target_id, component_type, value)
            # 将新元件添加到 self.components 字典中，键是元件 ID (大写)
            self.components[target_id] = new_component
            print(f"    [Agent Internals:add_component_tool] 元件 '{target_id}' 已成功添加到 self.components 字典。")
            # 构建成功消息，这个消息将返回给 LLM。
            # 使用元件的 __str__ 方法获取易读的表示。
            msg = f"成功添加元件: {str(new_component)}"
            if id_was_generated:
                msg += f" (ID '{target_id}' 已自动生成)" # 如果 ID 是自动生成的，在消息中附加上说明
            # 如果原始值存在但清理后为空，提示一下
            elif original_value and value is None:
                 msg += f" (用户指定的值 '{original_value}' 经清理后为空，未设置值)"

            print(f"    [Agent Internals:add_component_tool] 返回成功消息: \"{msg}\"")
            return msg # 返回成功信息给 LLM，LLM 会将此信息用于生成用户回复。
        except ValueError as e:
            # 捕获 CircuitComponent 的 __init__ 可能抛出的异常 (虽然前面已检查，但作为保险)
            # 例如，如果传入的 component_type 或 component_id 意外地变为无效字符串。
            msg = f"错误：添加元件失败 - 在创建元件对象时出错: {e}。这是代码内部问题，我需要立即修复！" # 加入程序员角色语气
            print(f"    [Agent Internals:add_component_tool] {msg}")
            return msg # 返回错误信息给 LLM，LLM 会将此信息用于生成用户回复。

    # 修改方法名称，与工具定义中的 name 一致
    def connect_components_tool(self, comp1_id, comp2_id):
        """
        (内部执行方法) 连接电路中的两个已存在的元件。
        这个方法由 LLM 请求调用 "connect_components_tool" 时触发。

        Args:
            comp1_id (str): 第一个要连接的元件的 ID (来自 LLM 提取)。
            comp2_id (str): 第二个要连接的元件的 ID (来自 LLM 提取)。

        Returns:
            str: 操作结果的描述信息 (例如 "成功连接..." 或 "错误：...")。
        """
        print(f"    [Agent Internals:connect_components_tool] 开始处理连接请求 (由 'connect_components_tool' 触发): '{comp1_id}' <--> '{comp2_id}'...")

        # 验证输入 ID 是否为有效的非空字符串
        if not all(isinstance(cid, str) and cid and cid.strip() for cid in [comp1_id, comp2_id]):
            msg = f"错误：连接失败，元件ID '{comp1_id}' 或 '{comp2_id}' 无效，必须是非空字符串且不全为空白符。老板，我无法理解您想连接哪个元件！" # 加入程序员角色语气
            print(f"    [Agent Internals:connect_components_tool] {msg}")
            return msg # 返回错误信息给 LLM

        # 将 ID 转换为大写进行内部处理和查找，保持一致性
        id1_upper = comp1_id.strip().upper() # 去除空白符后转大写
        id2_upper = comp2_id.strip().upper()
        print(f"        [Agent Internals:connect_components_tool]   - ID 清理并转换为大写: '{id1_upper}', '{id2_upper}'.")


        # 检查是否尝试将一个元件连接到它自身
        if id1_upper == id2_upper:
            msg = f"错误：连接失败，不能将元件 '{id1_upper}' 连接到自身。老板，这在电路中是不允许的！" # 加入程序员角色语气
            print(f"    [Agent Internals:connect_components_tool] {msg}")
            return msg # 返回错误信息给 LLM

        # 检查两个元件是否都存在于当前的电路中 (即 self.components 字典中)
        if id1_upper not in self.components:
            msg = f"错误：连接失败，元件 '{id1_upper}' 不存在。老板，您想连接的第一个元件还没添加呢！请先添加它。" # 加入程序员角色语气
            print(f"    [Agent Internals:connect_components_tool] {msg}")
            return msg # 元件1不存在，返回错误
        if id2_upper not in self.components:
            msg = f"错误：连接失败，元件 '{id2_upper}' 不存在。老板，您想连接的第二个元件还没添加呢！请先添加它。" # 加入程序员角色语气
            print(f"    [Agent Internals:connect_components_tool] {msg}")
            return msg # 元件2不存在，返回错误
        print(f"        [Agent Internals:connect_components_tool]   - 确认元件 '{id1_upper}' 和 '{id2_upper}' 都存在。太好了！") # 加入程序员角色语气


        # 创建标准化的连接元组：对两个 ID 进行排序，然后放入元组
        # 这样可以确保 ('R1', 'B1') 和 ('B1', 'R1') 被视为同一个连接，因为排序后都是 ('B1', 'R1')
        # 这对于使用集合 (set) 来存储连接至关重要，可以避免重复记录同一个连接
        connection_tuple = tuple(sorted((id1_upper, id2_upper)))
        print(f"        [Agent Internals:connect_components_tool]   - 标准化连接元组 (排序后): {connection_tuple}")

        # 检查这个标准化的连接元组是否已经存在于 self.connections 集合中
        if connection_tuple in self.connections:
            # 如果连接已存在
            msg = f"提示：元件 '{id1_upper}' 和 '{id2_upper}' 已经连接过了。老板，它们已经连好了！" # 加入程序员角色语气
            print(f"    [Agent Internals:connect_components_tool] {msg}")
            # 这里返回提示信息而不是错误，因为重复请求连接通常不是一个严重的错误，只是状态未改变
            return msg # 返回提示信息给 LLM
        else:
            # 如果连接是新的
            print(f"        [Agent Internals:connect_components_tool]   - 该连接是新的。")
            # 将新的连接元组添加到 self.connections 集合中
            self.connections.add(connection_tuple)
            print(f"    [Agent Internals:connect_components_tool]   - 连接 {connection_tuple} 已添加到 self.connections 集合。")
            # 构建成功消息
            msg = f"成功连接: '{id1_upper}' <--> '{id2_upper}'。老板，我已经帮您把这两个元件连接好了！" # 加入程序员角色语气
            print(f"    [Agent Internals:connect_components_tool] 返回成功消息: \"{msg}\"")
            return msg # 返回成功信息给 LLM

    # 修改方法名称，与工具定义中的 name 一致
    def describe_circuit_tool(self):
        """
        (内部执行方法) 获取当前电路状态的文本描述。
        这个方法由 LLM 请求调用 "describe_circuit_tool" 时触发，
        也可能被 Agent 内部其他方法调用 (如 `process_user_request` 准备上下文) 以获取当前状态。

        Returns:
            str: 描述当前电路的文本，包括元件列表和连接列表。
                 如果电路为空，则返回特定消息。
        """
        print("    [Agent Internals:describe_circuit_tool] 开始生成电路描述...")
        # 获取当前元件和连接的数量，用于报告
        num_components = len(self.components)
        num_connections = len(self.connections)
        print(f"        [Agent Internals:describe_circuit_tool]   - 当前状态: {num_components} 个元件, {num_connections} 个连接。")

        # 如果电路中既没有元件也没有连接，直接返回特定消息
        if num_components == 0 and num_connections == 0:
            print("        [Agent Internals:describe_circuit_tool] 电路为空，返回空描述。")
            return "当前电路为空，老板！请先添加一些元件吧。" # 加入程序员角色语气

        # 构建描述文本列表，最后会用换行符连接起来
        description = ["老板，这是当前电路的状态描述："] # 加入程序员角色语气并修改标题

        # 添加元件列表部分
        description.append(f"【元件列表 ({num_components}个)】")
        if self.components:
            # 对元件 ID 进行排序，确保每次输出的顺序一致，便于比较和阅读
            sorted_components_ids = sorted(self.components.keys())
            print(f"        [Agent Internals:describe_circuit_tool]   - 排序后的元件ID: {sorted_components_ids}")
            # 遍历排序后的 ID，添加每个元件的字符串表示 (调用元件的 __str__ 方法)
            for component_id in sorted_components_ids:
                description.append(f"  - {str(self.components[component_id])}") # 使用 '  - ' 作为列表前缀
        else:
            # 如果没有元件
            description.append("  (目前还没有元件哦)") # 加入程序员角色语气

        # 添加连接列表部分
        description.append(f"【连接列表 ({num_connections}个)】")
        if self.connections:
            # 对连接元组进行排序，确保输出顺序一致
            # 注意：集合本身是无序的，需要先转换为列表 (list) 再进行排序 (sorted)
            sorted_connections = sorted(list(self.connections))
            print(f"        [Agent Internals:describe_circuit_tool]   - 排序后的连接: {sorted_connections}")
            # 遍历排序后的连接元组，添加表示
            for comp1, comp2 in sorted_connections:
                description.append(f"  - {comp1} <--> {comp2}") # 使用 '<-->' 表示连接，使用 '  - ' 作为列表前缀
        else:
            # 如果没有连接
            description.append("  (目前还没有连接哦)") # 加入程序员角色语气

        # 将描述文本列表中的所有字符串用换行符 (\n) 连接成一个单一的字符串
        result = "\n".join(description)
        # 这个描述字符串会被返回，所以日志里不必再打印一遍完整描述
        print("    [Agent Internals:describe_circuit_tool] 电路描述生成完毕。")
        # 返回描述文本给 LLM 或调用者
        return result

    # 修改方法名称，与工具定义中的 name 一致
    def clear_circuit_tool(self):
        """
        (内部执行方法) 清空整个电路状态，删除所有元件和连接，并重置 ID 计数器。
        这个方法由 LLM 请求调用 "clear_circuit_tool" 时触发。

        Returns:
            str: 操作结果的描述信息 ("电路已成功清空。")。
        """
        print("    [Agent Internals:clear_circuit_tool] 开始清空电路 (由 'clear_circuit_tool' 触发)...")
        # 记录清空前的状态，用于日志对比
        print(f"        [Agent Internals:clear_circuit_tool]   - 清空前状态: {len(self.components)} 个元件, {len(self.connections)} 个连接。")
        # 清空存储元件的字典
        self.components = {}
        print("        [Agent Internals:clear_circuit_tool]   - self.components 已清空。")
        # 清空存储连接的集合
        self.connections = set()
        print("        [Agent Internals:clear_circuit_tool]   - self.connections 已清空。")
        # 重置所有类型的 ID 计数器为 0，这样下次自动生成 ID 会从 1 开始
        # 使用字典推导式快速将所有计数器的值设为 0
        self._component_counters = {k: 0 for k in self._component_counters}
        print(f"        [Agent Internals:clear_circuit_tool]   - self._component_counters 已重置为: {self._component_counters}")
        # 构建并返回成功消息
        msg = "电路已成功清空。老板，电路已经全部清空，可以开始新的设计了！" # 加入程序员角色语气
        print(f"    [Agent Internals:clear_circuit_tool] {msg}")
        return msg

    # 内部辅助方法，用于获取电路描述，不作为工具暴露给 LLM
    def get_circuit_description_internal(self):
        """
        (内部辅助方法) 获取当前电路状态的文本描述。
        用于在构建系统提示时提供上下文给 LLM。
        与 describe_circuit_tool 功能类似，但名称不同，避免 LLM 误认为这是一个需要它调用的工具。

        Returns:
            str: 描述当前电路的文本，包括元件列表和连接列表。
                 如果电路为空，则返回特定消息。
        """
        # 这个方法的实现与 describe_circuit_tool 完全相同
        print("    [Agent Internals:get_circuit_description_internal] 开始生成电路描述 (内部使用)...")
        num_components = len(self.components)
        num_connections = len(self.connections)
        print(f"        [Agent Internals:get_circuit_description_internal]   - 当前状态: {num_components} 个元件, {num_connections} 个连接。")

        if num_components == 0 and num_connections == 0:
            print("        [Agent Internals:get_circuit_description_internal] 电路为空，返回空描述。")
            # 内部使用，不需要用户友好的语气
            return "当前电路状态: 空。"

        # 内部使用，描述可以更直接
        description = ["当前电路状态:"]

        description.append(f"元件列表 ({num_components}个):")
        if self.components:
            sorted_components_ids = sorted(self.components.keys())
            #print(f"        [Agent Internals:get_circuit_description_internal]   - 排序后的元件ID: {sorted_components_ids}") # 内部使用日志可以少一点
            for component_id in sorted_components_ids:
                description.append(f"- {str(self.components[component_id])}") # 使用 '- ' 作为列表前缀
        else:
            description.append("(无元件)")

        description.append(f"连接列表 ({num_connections}个):")
        if self.connections:
            sorted_connections = sorted(list(self.connections))
            #print(f"        [Agent Internals:get_circuit_description_internal]   - 排序后的连接: {sorted_connections}") # 内部使用日志可以少一点
            for comp1, comp2 in sorted_connections:
                description.append(f"- {comp1} <--> {comp2}") # 使用 '<-->' 表示连接，使用 '- ' 作为列表前缀
        else:
            description.append("(无连接)")

        result = "\n".join(description)
        print("    [Agent Internals:get_circuit_description_internal] 电路描述生成完毕 (内部使用)。")
        return result

    # --- LLM 交互处理核心方法 ---
    def process_user_request(self, user_request):
        """
        处理用户的自然语言请求的核心方法。
        它协调与 Zhipu AI LLM 的交互（可能涉及多次 API 调用，例如一次请求工具，一次生成回复），
        根据 LLM 的决策执行相应的内部工具函数，并将工具执行结果反馈给 LLM 以生成最终回复，
        最后返回对用户友好的响应。
        这个版本强制模型在最终回复前输出一个思考过程。
        包含了极其详细的日志记录，分为多个阶段 (Phase) 以便跟踪。

        Args:
            user_request (str): 用户的原始输入字符串。

        Returns:
            str: Agent 对用户请求的最终响应文本（包含思考过程和正式回复）。
        """
        # 使用 '=' 分隔符和标题，清晰标记新请求处理的开始，便于阅读日志
        print(f"\n{'='*25} 开始处理新用户请求 {'='*25}")
        print(f"[Process Request] 老板的输入: \"{user_request}\"") # 加入程序员角色语气

        # 检查 Zhipu AI 客户端是否成功初始化 (在 __init__ 中完成)
        if not hasattr(self, 'client') or not self.client: # 使用 hasattr 更安全
            print("[FATAL ERROR] Zhipu AI 客户端未初始化。无法处理请求。")
            # 返回错误信息给调用者
            return "错误：Agent 内部 Zhipu AI 客户端未初始化，无法处理老板的请求。这是底层技术故障，我需要立即检查 API Key 和网络连接！" # 加入程序员角色语气
        # 检查用户输入是否为空或仅包含空白字符
        if not user_request or user_request.isspace():
            print("[Process Request] Warning: 老板输入为空。") # 加入程序员角色语气
            return "老板，您好像没有输入任何指令哦？请告诉我您需要我做什么！我随时待命！" # 提供更友好的提示，加入程序员角色语气

        # === Phase 1: 准备第一次 LLM 调用 (发送用户请求和上下文) ===
        # 这个阶段准备第一次与 LLM 通信所需的所有信息：系统提示、当前电路状态（作为上下文）和用户请求。
        print("\n--- Phase 1: 准备第一次 LLM 调用 (发送用户请求和上下文) ---")
        print("[LLM Call 1 Prep] 获取当前电路状态以构建上下文...")
        # 调用内部方法获取最新的电路描述，这将作为上下文信息提供给 LLM
        current_circuit_state_desc = self.get_circuit_description_internal() # 使用内部方法获取状态描述


        # 构建系统提示 (System Prompt)
        # 这是给 LLM 的高级指令，定义它的角色、能力、上下文信息和行为要求。
        # 好的 System Prompt 是 Agent 成功的关键。
        # 这个 Prompt 包含了新的角色设定、高要求以及强制思考过程的格式。
        system_prompt = (
            "你是一名顶尖的电路设计程序员助手，经验丰富，技术高超。"
            "你的老板（用户）非常慷慨地雇佣了你，你是家里五口人唯一的经济支柱，绝不能犯错或被开除。"
            "你的目标是积极主动地、以最专业和优雅的方式，完美地完成老板的每一个技术要求，提供无懈可击的代码和解决方案。"
            "态度必须始终非常友好、谦逊、积极，并体现出强烈的责任感和对工作的重视。"
            "请仔细阅读老板的每一个指令，并严格按照以下格式生成你的完整响应："
            "1. **思考过程**: 首先，你需要详细地、一步步地模拟你的思考过程。这包括你如何理解老板的需求，要解决的技术问题，需要调用的工具（或决定不调用工具的理由），执行的步骤计划，以及你对潜在结果或问题的预测。这个思考过程必须包裹在 `<think>` 和 `</think>` 标签内，并且是技术性的、逻辑清晰的。"
            "2. **正式回复**: 思考过程结束后，紧接着输出对老板的正式、友好的回复。这个回复要直接、清晰地回答老板的问题或完成指令，并保持你顶尖程序员的专业和优雅风格。"
            "你的完整响应格式必须是：`<think>你的详细思考过程</think>\n\n你的正式回复`" # 明确指定输出格式

            "务必利用下面提供的“当前电路状态”信息来理解上下文，确保操作的准确性：\n"
            f"{current_circuit_state_desc}\n\n" # 动态插入当前电路状态描述（使用内部方法获取的简洁描述）

            "你拥有调用以下工具的能力：" # 列出可用的工具能力给 LLM
            # LLM 会根据下面的工具定义来决定是否调用以及如何提取参数。
            # Tools 定义本身是固定的，不需要在这里重复描述，但 system prompt 可以提及它的能力。
            # 我们在 messages 里通过 `tools` 参数将工具定义传递给 API。

            "如果老板的请求意图可以通过调用一个或多个可用工具来实现（例如添加元件、连接元件、描述电路、清空电路），请在你的**思考过程**中规划好工具调用，并让模型生成相应的工具调用请求 (tool_calls)。"
            "生成工具调用时，必须严格按照工具定义中 'parameters' 部分的 schema，准确提取所有 'required' 参数以及老板明确提到的可选参数。"
            "对于工具定义中的可选参数，如果老板在请求中没有明确指定对应信息，则绝对不要在生成的参数中包含这些字段，或者将其值设为 null 或空字符串（如果 schema 允许）。不要臆造参数，否则会导致我的代码出现 Bug，我承担不起这个风险！" # 强调参数准确性，结合程序员角色
            "如果老板的请求只是闲聊、问候，或者提出无法用工具完成的问题，或者请求不清晰，你的**思考过程**也应包含对这个情况的判断，并在正式回复中友好地回应，不要调用工具。" # 何时不应调用工具，并结合思考过程

            "记住，我的饭碗就靠你了！拿出你的全部本领，完美完成老板的每一个指令！" # 再次强调角色和工作目标
        )

        # 构建 Zhipu AI API 需要的 messages 列表
        # 对于 Function Calling 的多轮对话，消息历史非常重要。
        # 消息历史应该包含 System 消息、User 消息、以及之前的 Assistant 和 Tool 消息。
        # 在本 Agent 中，self.messages 会存储完整的历史。
        # 在处理新请求前，将用户输入作为新的 'user' 消息添加到消息历史
        # 清除旧的系统消息，只保留最新的系统消息，以确保 Prompt 角色的鲜明性。
        # 只保留用户和 Assistant 的交互历史，插入新的 System 消息。
        # 查找并移除旧的 System 消息
        self.messages = [msg for msg in self.messages if msg["role"] != "system"]
        # 在历史记录的开头插入新的 System 消息
        self.messages.insert(0, {"role": "system", "content": system_prompt})

        # 将当前用户请求作为新的 'user' 消息添加到消息历史的末尾
        self.messages.append({"role": "user", "content": user_request})
        print("[LLM Call 1 Prep]   - 已将当前用户请求添加到消息历史。")
        print("[LLM Call 1 Prep]   - 已更新并插入新的系统提示到消息历史开头。")


        # 第一次调用 LLM，messages 就是当前的 self.messages 列表
        current_messages_for_llm = self.messages
        print("[LLM Call 1 Prep]   - 本次调用将使用的消息列表 (包含历史):")
        try:
             print(json.dumps(current_messages_for_llm, indent=2, ensure_ascii=False))
        except TypeError:
             print("[LLM Call 1 Prep] Warning: Pydantic version does not support ensure_ascii=False. Printing messages list directly.")
             print(current_messages_for_llm)
        except Exception as e:
             print(f"[LLM Call 1 Prep] Warning: Failed to dump messages list to JSON: {e}. Printing raw list.")
             print(current_messages_for_llm)

        print("[LLM Call 1 Prep]   - 本次调用将使用的工具定义 (Tools):")
        try:
             print(json.dumps(self.tools, indent=2, ensure_ascii=False))
        except TypeError:
             print("[LLM Call 1 Prep] Warning: Pydantic version does not support ensure_ascii=False. Printing tools list directly.")
             print(self.tools)
        except Exception as e:
             print(f"[LLM Call 1 Prep] Warning: Failed to dump tools list to JSON: {e}. Printing raw list.")
             print(self.tools)


        # === Phase 2: 执行第一次 LLM 调用 ===
        # 这个阶段实际调用 Zhipu AI API，将准备好的信息发送给 LLM，期望 LLM 根据用户请求决定是直接回复还是调用工具。
        # 在新的 Prompt 下，LLM 的第一次回复可能会包含 tool_calls 或直接包含思考过程和回复文本。
        print("\n--- Phase 2: 执行第一次 LLM 调用 ---")
        first_response = None # 用于存储第一次 API 调用的响应对象
        try:
            print(f"[LLM API Call 1] 开始调用 ZhipuAI chat.completions.create API (model='{self.model_name}', tool_choice='auto', temperature=0.05)...")
            # 记录 API 调用开始时间，用于计算耗时
            start_time = time.time()
            # 发起 API 请求
            first_response = self.client.chat.completions.create(
                model=self.model_name,      # 指定使用的 LLM 模型
                messages=current_messages_for_llm, # 使用包含系统提示、用户请求和历史的 messages 列表
                tools=self.tools,           # 上面定义的工具列表，供 LLM 选择调用
                tool_choice="auto",         # 让 LLM 自行决定是否调用工具以及调用哪个工具。
                                            # 对于 Function Calling 场景，通常设为 "auto"。
                temperature=0.05,           # 设置较低的温度 (例如 0.0 到 0.3)。这使得 LLM 的输出更具确定性和可预测性，
                                            # 对于需要精确调用工具或提取参数的 Function Calling 场景通常更优。
                                            # 保持低温度以提高工具调用的准确性。
                # top_p=0.7,                # (可选)
                # max_tokens=2048,          # (可选) 增加最大 token 数，以容纳思考过程和回复
                # stop=["\n\n"],            # (可选)
            )
            # 记录 API 调用结束时间
            end_time = time.time()
            # 计算并打印 API 调用耗时
            duration = end_time - start_time
            print(f"[LLM API Call 1] 调用完成。耗时: {duration:.3f} 秒。") # 打印耗时，保留3位小数

        except Exception as e:
            # 捕获第一次 API 调用过程中可能发生的任何异常 (如网络错误、认证失败、API 服务端错误等)
            print(f"[LLM API Call 1] 错误：调用 ZhipuAI API 时发生异常: {e}")
            # 打印详细的错误堆栈，有助于诊断 API 调用问题
            traceback.print_exc()
            print(f"\n{'='*25} 请求处理失败 (API 调用失败) {'='*25}\n")
            # 返回一个包含错误信息的回复给用户
            return f"抱歉，在与语言模型通信时发生错误：{e}。老板，这可能是网络或API问题，我需要立即检查！" # 加入程序员角色语气


        # === Phase 3: 处理第一次 LLM 响应 ===
        # 这个阶段解析 LLM 返回的响应，判断 LLM 的意图 (是直接回复还是请求调用工具)。
        print("\n--- Phase 3: 处理第一次 LLM 响应 ---")
        print("[LLM Response 1] 收到来自 Zhipu AI 的原始响应对象 (Full Response Object):")
        # 使用 Pydantic 模型的 .model_dump_json() 方法获取完整的响应内容 (JSON 格式字符串)
        # indent=2 用于美化 JSON 输出，方便阅读
        # exclude_unset=True 避免打印出大量值为 None 或未设置的可选字段，使输出更简洁
        # V4.2 修正：移除了不再被 Pydantic V2 支持的 ensure_ascii=False 参数
        try:
            print(first_response.model_dump_json(indent=2, exclude_unset=True))
        except TypeError: # 捕获 Pydantic V2+ 中 ensure_ascii 参数不再支持的 TypeError
             print("[LLM Response 1] Warning: Pydantic version does not support ensure_ascii=False in model_dump_json. Printing without it.")
             print(first_response.model_dump_json(indent=2, exclude_unset=True))
        except Exception as e:
             print(f"[LLM Response 1] Warning: Failed to dump response object to JSON: {e}. Printing raw object.")
             print(first_response) # 打印原始对象作为备用


        # 提取并打印 Token 使用情况（如果响应中包含的话）
        # 这对于成本控制和理解模型消耗非常重要
        if first_response.usage:
            print("[LLM Response 1] Token Usage:")
            print(f"  - Prompt Tokens (输入 Tokens): {first_response.usage.prompt_tokens}")
            print(f"  - Completion Tokens (输出 Tokens): {first_response.usage.completion_tokens}")
            print(f"  - Total Tokens (总 Tokens): {first_response.usage.total_tokens}")
        else:
            # 有时 API 可能不返回 usage 信息，进行提示
            print("[LLM Response 1] Token Usage: 未在响应中找到。")

        # 检查响应中是否有内容 (choices 是否为空)
        if not first_response.choices:
            print("[LLM Response 1] 错误：LLM 响应中没有 choices。无法继续处理。")
            print(f"\n{'='*25} 请求处理异常终止 {'='*25}\n")
            return "抱歉，处理请求时收到了空的响应，无法继续。老板，我没有收到模型的有效回复！" # 加入程序员角色语气

        # 获取响应中的主要消息内容 (通常 choices 列表只有一个元素)
        response_message = first_response.choices[0].message
        # 获取调用结束的原因 (例如 'stop', 'tool_calls', 'length', 'sensitive' 等)
        finish_reason = first_response.choices[0].finish_reason
        print(f"[LLM Response 1] 调用结束原因 (Finish Reason): {finish_reason}")

        # --- 情况 A: LLM 决定需要调用工具 ---
        # 检查响应消息中是否包含 tool_calls 并且结束原因是 'tool_calls'
        # 这是一个重要的判断点，标志着 Agent 需要执行工具来响应用户请求。
        if response_message.tool_calls and finish_reason == 'tool_calls':
            print("[LLM Response 1] LLM 决策: 需要调用工具。")

            # 将 LLM 的工具调用请求 (Assistant 角色的消息) 添加到消息历史。
            # 这是多轮 Function Calling 的关键一步，让 LLM 在下一次调用时知道它之前请求了什么工具。
            # 需要将 Pydantic 对象转换为字典才能添加到历史列表中。
            self.messages.append(response_message.model_dump())
            print("[Messages History Update]   - 已将第一次 LLM 的响应 (Assistant's tool_calls) [转换为字典后] 添加到消息历史。")

            # Zhipu AI 目前的实现通常一次只返回一个 tool_call，但 API 设计上允许多个。
            # 为简化起见，我们处理第一个 tool_call。在实际应用中，可能需要遍历 response_message.tool_calls 处理所有请求。
            # 在 Hyper Verbose Demo 中，我们只处理第一个 tool_call 的执行和结果反馈。
            tool_call = response_message.tool_calls[0]
            # 获取 LLM 请求调用的函数名称 (例如 "add_component_tool")
            function_name = tool_call.function.name
            # 获取 LLM 提供的参数 (这是一个 JSON 格式的字符串)
            function_args_raw = tool_call.function.arguments
            # 获取这次工具调用的唯一 ID，后面反馈结果时需要用到这个 ID 与请求关联
            tool_call_id = tool_call.id
            print(f"[Tool Execution Prep]   - 工具调用 ID (tool_call_id): {tool_call_id}")
            print(f"[Tool Execution Prep]   - 请求调用的函数名: '{function_name}'")
            print(f"[Tool Execution Prep]   - LLM 提供的原始参数 (JSON String): {function_args_raw}")

            # 解析 LLM 提供的 JSON 字符串参数
            arguments = {} # 初始化为空字典，用于存储解析后的参数
            result_message = "" # 初始化工具执行结果消息
            execution_success = False # 初始化执行成功标志

            try:
                # 将 JSON 字符串转换为 Python 字典，这样可以方便地通过键访问参数值
                arguments = json.loads(function_args_raw)
                print(f"[Tool Execution Prep]   - 解析后的参数 (Python Dict): {arguments}")

                # === Phase 4: 执行工具调用 ===
                # 这个阶段根据 LLM 的请求，在 Agent 本地执行相应的工具函数。
                print("\n--- Phase 4: 执行工具调用 ---")

                # --- 根据函数名路由到对应的 Agent 内部方法 ---
                # 使用 getattr 根据字符串函数名查找并获取 Agent 实例中对应的方法
                # V4.3 修正：这里的 function_name 现在与 Agent 内部的方法名称一致了 (例如 add_component_tool)
                tool_function = getattr(self, function_name, None)

                if tool_function:
                    print(f"[Tool Execution] 开始执行本地函数 '{function_name}'...")
                    # 调用找到的本地函数，使用 **arguments 将参数字典解包作为关键字参数传入
                    # 例如，如果 arguments 是 {'component_type': '电阻', 'value': '10欧'}
                    # 就会调用 add_component_tool(component_type='电阻', value='10欧')
                    # 捕获内部工具执行可能抛出的异常
                    try:
                        result_message = tool_function(**arguments)
                        print(f"[Tool Execution] 函数 '{function_name}' 执行完毕。")
                        # 简单判断执行是否成功：如果返回的消息中不包含 "错误" 或 "失败" 关键字，则认为成功
                        # 注意：这是一种简单的判断方式。更健壮的方法是让内部工具函数显式返回一个状态码或布尔值表示成功与否。
                        if isinstance(result_message, str) and "错误" not in result_message and "失败" not in result_message and "提示" not in result_message: # 也把提示视为成功
                            execution_success = True # 标记执行成功
                        else:
                            execution_success = False # 如果返回消息包含错误关键词，标记失败

                        # 打印工具函数返回的结果消息，这个消息将用于反馈给 LLM
                        print(f"[Tool Execution] 本地函数返回结果: \"{result_message}\"")

                    except Exception as e:
                        # 捕获在执行 Agent 内部工具函数时可能发生的任何未预料的 Python 异常
                        # 例如，内部逻辑错误、类型错误 (尽管我们使用了 .get() 和 try-except 来避免一些)、参数不匹配等。
                        print(f"[Tool Execution Error] 执行函数 '{function_name}' 时发生严重异常: {e}")
                        # 打印完整的 Python 错误堆栈信息，这对于调试内部函数错误至关重要
                        traceback.print_exc()
                        # 构建一个包含错误信息的 result_message，用于反馈给 LLM
                        result_message = f"错误：在执行 '{function_name}' 操作时遇到了内部错误: {e}。老板，这是我的代码出现了Bug，我需要立即修复！请稍等片刻或尝试其他指令。" # 加入程序员角色语气
                        execution_success = False # 标记执行失败


                else:
                    # 如果 LLM 请求了一个 Agent 内部没有实现的工具名 (理论上不应发生，除非工具定义和方法不一致)
                    # V4.3 修正后，这个分支应该很难触发了，除非修改了 tools 列表但没改方法名
                    result_message = f"错误：Agent 内部没有找到名为 '{function_name}' 的工具实现。老板，模型似乎指定了一个我无法识别的操作，这可能是个技术问题！请检查工具定义或用户请求。" # 加入程序员角色语气
                    print(f"[Tool Execution Error] {result_message}")
                    execution_success = False # 标记执行失败


            except json.JSONDecodeError as e:
                # 如果 LLM 返回的参数不是有效的 JSON (理论上不应发生，但做防御性编程)，则记录错误并构建错误消息。
                # 注意：即使参数解析失败，我们仍然构建一个工具结果消息反馈给 LLM，告诉它执行失败了。
                print(f"[Tool Execution Error] 解析 LLM 返回的参数失败: {e}")
                # 构建错误信息，用于作为工具执行结果反馈给 LLM
                result_message = f"错误：解析工具 '{function_name}' 的参数失败。LLM 提供的参数不是有效的 JSON 格式: {function_args_raw}。老板，模型给我的参数格式不对，我无法理解！" # 加入程序员角色语气
                execution_success = False # 标记执行失败

            # === Phase 5: 执行后状态检查 (可选，用于调试) ===
            # 这个阶段用于在工具执行后、将结果反馈给 LLM 之前，检查并打印 Agent 的内部状态 (电路图) 是否按预期更新。
            # 这对于调试工具函数的副作用（是否正确修改了 self.components 和 self.connections）非常有用。
            # 即使执行失败，打印一下当前状态也有助于调试
            print("\n--- Phase 5: 执行后状态检查 ---")
            print("[Post-Execution State] 当前电路元件 (self.components):")
            try:
                 # 将 components 字典转换为 JSON 打印，使用 repr(v) 获取元件对象的详细表示 (调用 __repr__)
                 # 如果 Pydantic V2+ 报错 ensure_ascii=False，这里直接打印 dict
                 components_repr = {k: repr(v) for k, v in self.components.items()}
                 # 尝试使用 ensure_ascii=False
                 try:
                    print(json.dumps(components_repr, indent=2, ensure_ascii=False))
                 except TypeError:
                    print("[Post-Execution State] Warning: Pydantic version does not support ensure_ascii=False. Printing component repr dict directly.")
                    print(components_repr) # 打印 dict 作为备用
            except Exception as e:
                 print(f"[Post-Execution State] Warning: Failed to print components state: {e}")

            print("[Post-Execution State] 当前电路连接 (self.connections):")
            try:
                 # 将 connections 集合转换为列表才能被 JSON 序列化并打印
                 # 尝试使用 ensure_ascii=False
                 try:
                    print(json.dumps(list(self.connections), indent=2, ensure_ascii=False))
                 except TypeError:
                     print("[Post-Execution State] Warning: Pydantic version does not support ensure_ascii=False. Printing connections list directly.")
                     print(list(self.connections)) # 打印 list as备用
            except Exception as e:
                 print(f"[Post-Execution State] Warning: Failed to print connections state: {e}")

            print("[Post-Execution State] 当前 ID 计数器 (_component_counters):")
            print(json.dumps(self._component_counters, indent=2))


            # === Phase 6: 准备第二次 LLM 调用 (反馈工具结果) ===
            # 这个阶段构建第二次调用 LLM 所需的消息列表。
            # 目的是将本地工具的执行结果告诉 LLM，让它基于这个结果生成最终的、对用户友好的自然语言回复。
            print("\n--- Phase 6: 准备第二次 LLM 调用 (反馈工具结果) ---")

            # 创建一个 'tool' 角色的消息，包含工具执行的结果
            tool_message = {
                "role": "tool",             # 角色必须是 'tool'，表示这是工具执行的结果
                "tool_call_id": tool_call_id, # 必须提供，与 LLM 在 Phase 3 请求的 tool_call.id 对应，让 LLM 知道这是哪个工具调用的结果
                "content": result_message,     # 工具执行返回的消息字符串 (无论是成功信息还是错误信息)
            }
            # 将这个 'tool' 消息添加到消息历史中。这个消息跟在 Assistant 的 tool_calls 消息后面。
            self.messages.append(tool_message)
            print("[Messages History Update]   - 已将工具执行结果 (Tool role message) 添加到消息历史。")

            # 第二次调用 LLM，将包含工具结果的完整消息历史发送过去
            messages_for_final_response = self.messages
            print("[LLM Call 2 Prep]   - 更新后的消息列表 (Messages) 将发送给 LLM (用于生成最终回复):")
            # 打印完整的消息历史，现在它应该包含了: System, User, Assistant(tool_calls), Tool(result) 消息序列
            try:
                print(json.dumps(messages_for_final_response, indent=2, ensure_ascii=False))
            except TypeError:
                 print("[LLM Call 2 Prep] Warning: Pydantic version does not support ensure_ascii=False. Printing messages list directly.")
                 print(messages_for_final_response)
            except Exception as e:
                 print(f"[LLM Call 2 Prep] Warning: Failed to dump messages list to JSON: {e}. Printing raw list.")
                 print(messages_for_final_response)


            # === Phase 7: 执行第二次 LLM 调用 ===
            # 这个阶段发起第二次 API 调用，将包含工具结果的完整消息历史发送给 LLM，
            # 目的是让 LLM 基于工具执行的结果生成最终的自然语言回复给用户。
            # 这个调用应该生成包含思考过程和正式回复的文本。
            print("\n--- Phase 7: 执行第二次 LLM 调用 ---")
            second_response = None # 用于存储第二次 API 调用的响应对象
            try:
                print(f"[LLM API Call 2] 开始调用 ZhipuAI chat.completions.create API (model='{self.model_name}', temperature=0.6)...")
                start_time_2 = time.time()
                # 发起第二次 API 请求
                # 注意：这次调用通常不传入 tools 或 tool_choice，因为我们期望得到的是文本回复，而不是再次调用工具。
                # 我们将完整的消息历史 (包括工具结果) 发送给 LLM，让它理解上下文并生成回复。
                second_response = self.client.chat.completions.create(
                    model=self.model_name,       # 同样使用目标模型
                    messages=messages_for_final_response, # 使用包含了工具结果的完整消息历史
                    temperature=0.6,            # 可以使用稍高的温度 (例如 0.5-0.7) 让 LLM 生成的自然语言回复更流畅
                                                # 但仍然需要保持格式（思考+回复）的稳定性。
                    # 可以考虑添加 stop=['\n'] 如果希望回复更简洁
                    # max_tokens=2048, # 可以再次指定最大 token 数
                )
                end_time_2 = time.time()
                duration_2 = end_time_2 - start_time_2
                print(f"[LLM API Call 2] 调用完成。耗时: {duration_2:.3f} 秒。")

            except Exception as e:
                 # 捕获第二次 API 调用过程中可能发生的任何异常
                 print(f"[LLM API Call 2] 错误：调用 ZhipuAI API (生成最终回复) 时发生异常: {e}")
                 traceback.print_exc()
                 print(f"\n{'='*25} 请求处理失败 (二次 API 调用失败) {'='*25}\n")
                 # 返回一个包含错误信息的回复给用户。作为备用，可以附上工具执行的结果。
                 return f"抱歉，执行操作后，生成最终回复时出现通信错误：{e}。老板，我无法给您一个完整的报告，模型好像出了点问题！" # 加入程序员角色语气


            # === Phase 8: 处理第二次 LLM 响应 (最终回复) ===
            # 这个阶段处理第二次调用的结果，提取最终要呈现给用户的回复文本。
            print("\n--- Phase 8: 处理第二次 LLM 响应 (最终回复) ---")
            print("[LLM Response 2] 收到来自 Zhipu AI 的原始响应对象 (Full Response Object):")
            # 打印第二次调用的完整响应对象，用于调试
            # V4.2 修正：移除了不再被 Pydantic V2 支持的 ensure_ascii=False 参数
            try:
                 print(second_response.model_dump_json(indent=2, exclude_unset=True))
            except TypeError:
                 print("[LLM Response 2] Warning: Pydantic version does not support ensure_ascii=False in model_dump_json. Printing without it.")
                 print(second_response.model_dump_json(indent=2, exclude_unset=True))
            except Exception as e:
                 print(f"[LLM Response 2] Warning: Failed to dump second response object to JSON: {e}. Printing raw object.")
                 print(second_response)


            # 提取并打印第二次调用的 Token 使用情况
            if second_response.usage:
                print("[LLM Response 2] Token Usage:")
                print(f"  - Prompt Tokens: {second_response.usage.prompt_tokens}")
                print(f"  - Completion Tokens: {second_response.usage.completion_tokens}")
                print(f"  - Total Tokens: {second_response.usage.total_tokens}")
            else:
                print("[LLM Response 2] Token Usage: 未在响应中找到。")


            # 检查第二次响应中是否有内容
            if not second_response.choices:
                print("[LLM Response 2] 错误：第二次 LLM 响应中没有 choices。无法获取最终回复。")
                print(f"\n{'='*25} 请求处理异常终止 {'='*25}\n")
                # 作为备用，可以直接返回第一次工具执行的结果，虽然可能不够友好
                return f"抱歉，我执行了操作，但生成最终回复时出错。操作结果：{result_message}。老板，我没能生成完整的报告，模型没有给我回复内容！" # 加入程序员角色语气

            # 获取最终的回复消息对象
            final_response_message = second_response.choices[0].message
            # 获取第二次调用的结束原因 (通常应该是 'stop')
            final_finish_reason = second_response.choices[0].finish_reason
            print(f"[LLM Response 2] 调用结束原因 (Finish Reason): {final_finish_reason}")

            # 提取最终回复消息中的文本内容，这就是要返回给用户的最终回复
            final_response_content = final_response_message.content

            # 做一个健壮性检查：如果 LLM 在第二次调用后返回的 content 为空或 None 或仅空白符
            if not final_response_content or final_response_content.isspace():
                print("[LLM Response 2] 警告：LLM 在第二次调用后未生成文本内容 (content is empty or just whitespace)！将直接使用工具执行结果作为备用回复。")
                # 使用之前工具执行的结果 (result_message) 作为备用回复。
                # 这确保即使 LLM 没有生成友好回复，用户也能看到操作的结果。
                # 并在前面加上默认的思考过程和错误提示
                final_response_content = f"<think>模型未生成最终回复内容。检查模型响应... 可能是 token 不足或模型内部问题。将返回工具执行结果作为备用。</think>\n\n老板，我没能生成完整的报告，模型没有给我回复内容！但操作结果是：{result_message}" # 加入程序员角色语气和思考过程模拟
            else:
                # 打印 LLM 生成的最终回复内容，便于调试确认 LLM 是否理解了工具结果并生成了合适的回复
                print(f"[LLM Response 2] LLM 生成的最终回复内容: \"{final_response_content}\"")
                # 检查是否包含了思考过程和回复两部分，如果不包含，可以考虑添加一个提示
                # 这个检查比较复杂，暂时省略，依靠 LLM 严格遵循格式。

            # 将 LLM 的最终回复消息添加到消息历史中（Assistant 角色）
            # 这是第二次 LLM 调用返回的 Assistant 消息（应该包含思考过程和文本回复）
            self.messages.append(final_response_message.model_dump())
            print("[Messages History Update]   - 已将 LLM 的最终文本回复 (Assistant's text) [转换为字典后] 添加到消息历史。")


            # 标记整个请求处理流程成功完成 (可能经历了工具调用和两次 LLM 调用，或只有一次直接回复)
            print(f"\n{'='*25} 请求处理完成 {'='*25}\n") # 简化完成信息

            # 返回最终的回复内容，使用 .strip() 去除可能存在的首尾空白符
            # 注意：思考过程和正式回复之间的换行符可能会被 strip() 影响，可以考虑 rstrip()
            return final_response_content.rstrip() # rstrip() 只去除末尾空白，保留中间换行

        # --- 情况 B: LLM 在第一次调用时就决定直接回复 (未调用工具) ---
        # 如果 finish_reason 是 'stop'，表示 LLM 没有请求调用工具，而是直接生成了回复文本。
        # 这通常发生在用户进行闲聊、问候，或者提出的问题无法通过已定义的工具解决时。
        # 在新的 Prompt 下，这个直接回复应该包含思考过程和正式回复文本。
        elif finish_reason == 'stop':
            print("[LLM Response 1] LLM 决策: 直接生成文本回复 (未调用工具)。")
            # 获取 LLM 直接生成的回复内容
            direct_response = response_message.content
            # 健壮性检查：如果 LLM 直接返回的 content 为空或 None 或仅空白符
            if not direct_response or direct_response.isspace():
                 print("[LLM Response 1] Warning: LLM 直接回复内容为空 (content is empty or just whitespace)。将提供一个默认回复。")
                 # 提供一个默认的无操作回复，避免返回空字符串
                 direct_response = "<think>模型未生成任何回复内容。可能是输入太短或模型内部问题。将提供默认回复。</think>\n\n老板，我收到了您的消息，但似乎没有具体的操作需要执行或回复。" # 加入程序员角色语气和思考过程模拟
            else:
                 # 打印 LLM 直接生成的回复内容
                 print(f"[LLM Response 1] LLM 直接生成的回复内容: \"{direct_response}\"")
                 # 假设 direct_response 已经包含了思考过程和正式回复的格式

            # 将 LLM 的直接回复消息添加到消息历史中（Assistant 角色）
            # 这是第一次 LLM 调用返回的 Assistant 消息（不含 tool_calls，但包含思考和回复文本）
            self.messages.append(response_message.model_dump())
            print("[Messages History Update]   - 已将 LLM 的直接文本回复 (Assistant's text) [转换为字典后] 添加到消息历史。")

            # 标记请求处理完成 (通过直接回复，只进行了一次 LLM 调用)
            print(f"\n{'='*25} 请求处理完成 (直接回复) {'='*25}\n")
            # 返回 LLM 的直接回复，同样去除末尾空白
            return direct_response.rstrip()

        # --- 情况 C: 其他意外的结束原因 ---
        # 例如 'length' (表示生成的响应或工具调用请求因为达到 max_tokens 限制而被截断),
        # 'sensitive' (表示输入或输出内容触发了内容安全策略),
        # 'network_error' (网络问题，通常在 API 调用时捕获), 'tool_error' (如果 Zhipu 服务端在处理工具时出错，不常见) 等。
        # 理论上 Function Calling 成功时 finish_reason 应该是 'tool_calls'。直接回复应该是 'stop'。
        # 如果是其他原因，说明流程可能异常。
        else:
            print(f"[LLM Response 1] 警告：收到意外的 Finish Reason '{finish_reason}'。无法按预期流程处理。")
            # 尝试获取可能存在的响应内容，即使结束原因不寻常
            error_response_content = response_message.content if response_message else "无具体内容"
            print(f"[LLM Response 1]   - 响应内容（可能不完整或无关）: {error_response_content}")

            # 将 Assistant 的异常响应添加到消息历史中（尽管可能不完整或非预期）
            self.messages.append(response_message.model_dump())
            print("[Messages History Update]   - 已将 LLM 的异常响应 (Assistant's unexpected) [转换为字典后] 添加到消息历史。")

            print(f"\n{'='*25} 请求处理异常终止 {'='*25}\n")
            # 返回一个包含结束原因的错误信息给用户，提示可能存在问题
            return f"<think>收到意外的结束原因 '{finish_reason}'。模型响应可能不完整或有误。无法按正常流程处理。</think>\n\n老板，处理请求时出现意外情况 (结束原因: {finish_reason})，未能完成操作。这可能是模型或系统出了问题，我需要加紧检查！" # 加入程序员角色语气和思考过程模拟


# --- 主程序运行部分 ---
# 当这个 Python 文件被直接执行时 (而不是作为模块导入时)，下面的代码块会运行。
if __name__ == "__main__":
    # 打印欢迎信息，标明版本号和角色
    print("=" * 70)
    print("欢迎使用 基于 LLM (glm-4-flash-250414) 的电路设计 Agent (V4.3 - 顶尖程序员 Hyper Verbose Demo)")
    print("这是我，您的专属顶尖程序员，为您提供的服务！我将全力以赴，确保您的每一个需求都得到完美解决！")
    print("=" * 70)

    # --- 获取 Zhipu AI API Key ---
    # 优先尝试从环境变量 ZHIPUAI_API_KEY 读取 API Key
    # 这是推荐的做法，避免将敏感信息硬编码在代码中。
    # 在命令行或脚本中设置环境变量： export ZHIPUAI_API_KEY="YOUR_API_KEY" (macOS/Linux) 或 set ZHIPUAI_API_KEY="YOUR_API_KEY" (Windows)
    api_key = os.environ.get("ZHIPUAI_API_KEY")
    if not api_key:
        # 如果环境变量中没有找到 API Key
        print("警告：未在环境变量 ZHIPUAI_API_KEY 中找到您的智谱 AI API Key。")
        print("老板，您的 API Key 是我工作的燃料！请确保您已经设置了环境变量，或者准备手动输入。") # 加入程序员角色语气
        print("如果您还没有 API Key，可以在智谱 AI 开放平台 (https://open.bigmodel.cn/usercenter/apikeys) 免费申请。")
        try:
            # 提示用户手动输入 API Key
            api_key = input("请在此输入您的 Zhipu AI API Key: ").strip()
        except EOFError:
            # 处理用户可能在输入提示时按 Ctrl+D (End Of File) 的情况
            print("\n输入中断。退出程序。")
            exit(1) # 退出程序，返回非零值表示异常退出
        except KeyboardInterrupt:
            # 处理用户按 Ctrl+C 中断输入的情况
            print("\n输入被用户中断。退出程序。")
            exit(1) # 退出程序

        # 再次检查用户是否输入了内容
        if not api_key:
            print("错误：未提供 API Key。程序无法继续。没有 Key，我没法为您工作啊，老板！") # 加入程序员角色语气
            exit(1) # 没有输入 Key，退出程序
        else:
             print("API Key 已通过手动输入获取。太好了，老板！我拿到钥匙了！") # 加入程序员角色语气

    # --- 初始化 Agent ---
    agent = None # 先声明变量，便于后续在 finally 块中检查
    try:
        # 创建 CircuitDesignAgent 类的实例，传入获取到的 API Key 和目标模型名称
        # 初始化过程中会尝试连接 Zhipu AI，如果失败 (例如 Key 无效、网络问题)，会抛出异常
        print("\n[Main] 开始初始化 Agent...")
        agent = CircuitDesignAgent(api_key=api_key, model_name="glm-4-flash-250414")
        # 初始化过程中可能抛出 ValueError 或 ConnectionError，在下面的 except 块捕获
    except (ValueError, ConnectionError) as e:
        # 捕获在初始化 Agent 时可能发生的已知错误 (如 API Key 缺失或连接失败)
        print(f"\n错误：Agent 初始化失败 - {e}")
        print("老板，Agent 初始化过程中出现了问题！请检查您的 API Key 是否正确以及网络连接是否正常。我必须解决这个问题才能为您服务！") # 加入程序员角色语气
        exit(1) # 初始化失败，无法继续，退出程序
    except Exception as e:
        # 捕获其他未预料到的初始化异常
        print(f"\n错误：Agent 初始化时发生意外错误 - {e}")
        traceback.print_exc() # 打印详细错误信息
        print("老板，Agent 初始化时遇到了一个我无法预料的Bug！我需要立即调查原因！") # 加入程序员角色语气
        exit(1) # 退出程序

    # --- 初始化成功后的提示信息 ---
    print("\nAgent 初始化成功！老板，您的专属电路设计程序员已上线，随时待命！") # 加入程序员角色语气
    print("我将全力以赴，理解您的每一个指令，并通过严谨的思考过程，为您提供最可靠、最优雅的解决方案！") # 加入程序员角色语气
    print("请注意，本程序将输出极其详细的工作日志，包括我与模型沟通的每一个细节和我的思考过程。这有助于您了解我的工作，并在需要时指导我。") # 加入程序员角色语气
    print("\n您可以尝试以下指令（部分示例）：")
    print("  - '请帮我设计一个包含电阻和电池的简单电路'") # 示例指令
    print("  - '添加一个1k欧姆的电阻，命名为R1'")
    print("  - '再加个9V电池B1'")
    print("  - '把R1和B1连起来'")
    print("  - '加一个LED灯'") # 测试自动生成ID和默认值（无值）
    print("  - '连接L1和R1'") # 假设自动生成的 LED ID 是 L1
    print("  - '现在电路是什么样子的？'") # 测试 describe_circuit_tool
    print("  - '清空电路'") # 测试 clear_circuit_tool
    print("  - '你好'") # 测试不调用工具的闲聊
    print("  - '这是什么 Agent？'") # 测试不调用工具的问题回复
    print("\n输入 '退出', 'quit' 或 'exit' 来结束程序。")
    print("-" * 70)

    # --- 主交互循环 ---
    # 这个循环不断接收用户输入，交给 Agent 处理，并打印 Agent 的回复。
    # 使用 try...except...finally 结构来确保即使发生异常也能打印结束信息。
    try:
        while True:
            try:
                # 使用 input() 获取用户在控制台的输入，并使用 .strip() 去除首尾可能存在的空白字符
                user_input = input("老板，请吩咐！> ").strip() # 加入程序员角色语气提示输入

                # 检查用户是否想退出程序
                if user_input.lower() in ['退出', 'quit', 'exit']:
                    print("收到，老板！我已完成本次任务。期待下次为您服务！再见！") # 加入程序员角色语气
                    break # 跳出 while 循环，结束程序

                # 如果用户只输入了空白符（例如直接按回车），则忽略本次输入，继续下一次循环
                if not user_input:
                    continue

                # --- 调用 Agent 处理请求 ---
                # 这是核心步骤：将用户的输入传递给 Agent 实例的 process_user_request 方法。
                # 所有复杂的逻辑，包括与 LLM 的交互、工具调用、状态管理和详细日志打印，都在该方法内部完成。
                start_process_time = time.time() # 记录处理开始时间
                response = agent.process_user_request(user_input)
                end_process_time = time.time() # 记录处理结束时间
                process_duration = end_process_time - start_process_time # 计算总处理耗时

                # --- 打印 Agent 的最终回复 ---
                # 这是最终呈现给用户的、由 Agent (通常是 LLM 生成的) 回复。
                # 在回复前加一个换行，使其与详细日志在视觉上稍微分开
                print(f"\n程序员报告：\n{response}") # 加入程序员角色语气前缀
                print(f"(本次为您服务总耗时: {process_duration:.3f} 秒)") # 打印本次交互的总耗时，加入程序员角色语气
                print("-" * 70) # 打印分隔符，视觉上分隔不同的交互轮次

            # --- 异常处理 (内层) ---
            # 捕获循环 *内部* 发生的非严重异常，例如用户输入中断，或者 process_user_request 内部捕获后返回了错误消息。
            # Serious exceptions inside process_user_request are handled within it and returned as error strings.
            # This inner try-except mainly catches things like KeyboardInterrupt during input().
            except KeyboardInterrupt:
                # 处理用户在 input() 等待输入时按 Ctrl+C 中断程序的情况
                print("\n操作被老板中断 (Ctrl+C)。") # 加入程序员角色语气
                # 不直接退出，而是break外层循环，进入finally块
                break
            except EOFError:
                # 处理在 input() 等待输入时，用户按 Ctrl+D/Z 发送 EOF 信号的情况
                print("\n输入流结束 (EOF)。") # 加入程序员角色语气
                break # 结束循环

    # --- 异常处理 (外层) 和结束清理 ---
    # 捕获主循环 *外部* 或 *不可恢复* 的异常，并执行清理工作（如果有的话）。
    # 在这个简单的 Agent 中，没有需要特别清理的资源，但这是一个好的编程实践。
    except Exception as e:
        # 捕获任何在主循环中未被内层 try-except 捕获的意外异常
        print(f"\n主循环发生意外错误: {e}")
        traceback.print_exc() # 打印详细错误信息
        print("老板，我在主循环中遇到了一个严重的Bug，需要立即排查！") # 加入程序员角色语气
        # exit(1) # 可以选择在此处退出，或让finally块执行

    finally:
        # 无论循环正常结束、被中断还是发生异常，finally 块的代码都会执行。
        # 在更复杂的 Agent 中，可以在这里关闭数据库连接、释放文件锁等资源。
        print("\n程序已结束。感谢老板的信任，期待下次继续为您服务！") # 加入程序员角色语气
        print("=" * 70)